## This notebook is for fetching data, if you have the data "binance_30m_data.csv" locally, directly go to cryptocurrency_analysis

In [1]:
print('\033[32mJUNXUN LIU 1005836259\033[0m')

JUNXUN LIU 1005836259


In [1]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

C:\Users\devin\anaconda3\envs\py39\python.exe
3.9.16 (main, Mar  8 2023, 10:39:24) [MSC v.1916 64 bit (AMD64)]
sys.version_info(major=3, minor=9, micro=16, releaselevel='final', serial=0)


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import pytz
import datetime
try:
    import ccxt
except ImportError:
    ! pip install ccxt
    import talib
    import ccxt
import time

pd.set_option('display.max_rows', 5000)
pd.set_option('expand_frame_repr', False)

In [6]:
currency_lst = ['ETH/USDT', 'DOGE/USDT', 'SOL/USDT', 'FIL/USDT', 'LTC/USDT', 'BNB/USDT', 'SHIB/USDT', 'XRP/USDT', 'AAVE/USDT', ]

In [7]:
# create the exchange object
exchange = ccxt.binance()
# set the timeframe to 1 minute
timeframe = '30m'
# set the start time to 1 month before
start_time = int(datetime.datetime.strptime("2022-01-01 00:00:00", "%Y-%m-%d %H:%M:%S").timestamp() * 1000)
# set the end time to the current time
end_time = int(datetime.datetime.strptime("2023-01-01 00:00:00", "%Y-%m-%d %H:%M:%S").timestamp() * 1000)
# set the number of data points to fetch per API request
limit = 1000

In [8]:
# create an empty dataframe to hold the results
df_btc = pd.DataFrame()

In [9]:
# loop through the time range and fetch data
while start_time < end_time:
    # fetch the OHLCV data
    ohlcv_btc = exchange.fetch_ohlcv('BTC/USDT', timeframe, since=start_time, limit=limit)
    # convert the data to a pandas dataframe
    df = pd.DataFrame(ohlcv_btc, columns=['timestamp', 'open', 'high', 'low', 'BTC_close', 'BTC_volume'])
    # append the data to the main dataframe
    df_btc = pd.concat([df_btc, df])
    # set the start time for the next request
    start_time = int(df['timestamp'].iloc[-1]) + 60 * 1000

In [10]:
# convert the timestamp to datetime format and set timezone Toronto time
df_btc['timestamp'] = pd.to_datetime(df_btc['timestamp'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('America/Toronto').dt.strftime('%Y-%m-%d %H:%M:%S')

In [11]:
# set the timestamp as the index
df_btc.set_index('timestamp', inplace=True)

In [12]:
# get close price and volume for BTC
df_btc_px = df_btc['BTC_close']
df_btc_vol = df_btc['BTC_volume']
df = pd.merge(df_btc_px, df_btc_vol, on='timestamp')

In [13]:
df

,BTC_close,BTC_volume
timestamp,,
2022-01-01 00:00:00,47379.13,793.08192
2022-01-01 00:30:00,47192.55,607.65450
2022-01-01 01:00:00,47130.36,267.90481
2022-01-01 01:30:00,46979.62,346.07295
2022-01-01 02:00:00,46959.61,295.25750
...,...,...
2023-01-10 21:30:00,17388.20,4010.98004
2023-01-10 22:00:00,17410.69,4549.10986
2023-01-10 22:30:00,17416.14,2853.62609


#### Get data for rest 9 cryptos, please wait for API to fetch data, it may takes 2 minutes to get 10 cryptos data. This is because when request data from binance, it only gives 1000 data once (OKX is 100). Should use while loop to get full data.

In [14]:
for crypto in currency_lst:
    cur = crypto[:-5]
    close = cur + '_close'
    volume = cur + '_volume'
    start_time = int(datetime.datetime.strptime("2022-01-01 00:00:00", "%Y-%m-%d %H:%M:%S").timestamp() * 1000)
    df_crypto = pd.DataFrame()
    while start_time < end_time:
        # fetch the OHLCV data
        ohlcv_temp = exchange.fetch_ohlcv(crypto, timeframe, since=start_time, limit=limit)
        # convert the data to a pandas dataframe
        df_temp = pd.DataFrame(ohlcv_temp, columns=['timestamp', 'open', 'high', 'low', close, volume])
        # append the data to the main dataframe
        df_crypto = pd.concat([df_crypto, df_temp])
        # set the start time for the next request
        start_time = int(df_temp['timestamp'].iloc[-1]) + 60 * 1000
    df_crypto['timestamp'] = pd.to_datetime(df_crypto['timestamp'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('America/Toronto').dt.strftime('%Y-%m-%d %H:%M:%S')
    # set the timestamp as the index
    df_crypto.set_index('timestamp', inplace=True)
    df_crypto_px = df_crypto[close]
    df_crypto_vol = df_crypto[volume]
    df_merge = pd.merge(df_crypto_px, df_crypto_vol, on='timestamp')
    df = pd.merge(df, df_merge, on='timestamp')

In [15]:
# Use exactly 1 moth data
df = df.loc['2022-01-01 00:00:00':'2023-01-01 00:00:00']

In [16]:
df.to_csv("data/binance_30m_data.csv", index=True)